# Interaction Sequences: TNFR-friendly telemetry and export

Este cuaderno genera secuencias de interacción reproducibles, telemetría sintética y artefactos (figuras CSV/JSON) y escribe una guía en docs/INTERACTIONS_GUIDE.md. Sigue los principios TNFR: prioriza coherencia, trazabilidad y reproducibilidad. Las figuras y tablas se exportan a docs/assets/interactions/.

Requisitos clave:
- Semilla global fija para reproducibilidad
- Exportar manifest con rutas y metadatos
- Validar invariantes simples de las series (no-negatividad, timestamps monótonos)
- Generar una guía Markdown con las imágenes y enlaces a tablas

## 1) Configurar entorno y rutas de proyecto

Detectamos la raíz del repositorio (git o heurística con pathlib), creamos carpetas de salida y parametrizamos rutas para activos (PNG/SVG/CSV/JSON).

from __future__ import annotations
import os
import sys
from pathlib import Path
from typing import Dict, Any

# Detect repo root via git, else fallback to current working directory
try:
    import subprocess
    root_guess = Path.cwd()
    git_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], cwd=root_guess).decode().strip()
    REPO_ROOT = Path(git_root)
except Exception:
    # Heurística: buscar pyproject.toml hacia arriba
    p = Path.cwd()
    while p != p.parent:
        if (p / "pyproject.toml").exists():
            REPO_ROOT = p
            break
        p = p.parent
    else:
        REPO_ROOT = Path.cwd()

# Ensure src is importable during interactive runs
SRC_DIR = REPO_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

DOCS_DIR = REPO_ROOT / "docs"
ASSETS_DIR = DOCS_DIR / "assets" / "interactions"
DATA_DIR = REPO_ROOT / "data"
REPORTS_DIR = REPO_ROOT / "results" / "reports"

for d in [DOCS_DIR, ASSETS_DIR, DATA_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Assets dir: {ASSETS_DIR}")

## 2) Importar librerías y establecer semilla

Importamos librerías mínimas y fijamos semilla global para reproducibilidad.

import json
import yaml
import math
import hashlib
import platform
import random
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

try:
    # Optional: richer version reporting
    import pkg_resources  # type: ignore
except Exception:
    pkg_resources = None

# Fixed seeds for reproducibility
SEED = int(os.environ.get("INTERACTIONS_SEED", 1337))
np.random.seed(SEED)
random.seed(SEED)

print({"numpy_seed": SEED})

## 3) Cargar o simular telemetría base

Implementamos carga desde CSV/JSON en data/ y un simulador de series sintéticas (tiempo, latencia, throughput, errores) con ruido controlado por la semilla.

def load_telemetry(path: Path) -> pd.DataFrame:
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(path)
    if path.suffix.lower() == ".csv":
        return pd.read_csv(path)
    if path.suffix.lower() in {".json", ".jsonl"}:
        if path.suffix.lower() == ".jsonl":
            rows = [json.loads(line) for line in path.read_text(encoding="utf-8").splitlines() if line.strip()]
            return pd.DataFrame(rows)
        return pd.DataFrame(json.loads(path.read_text(encoding="utf-8")))
    raise ValueError(f"Unsupported telemetry format: {path.suffix}")


def simulate_telemetry(n: int = 300, base_latency_ms: float = 120.0, base_rps: float = 50.0,
                        error_p: float = 0.01, spike_every: int | None = 60,
                        duration_s: int = 300, step_label: str = "base",
                        start_ts: datetime | None = None) -> pd.DataFrame:
    """Generate synthetic telemetry with gentle trends and occasional spikes.
    Columns: ts, t_s, step, latency_ms, throughput_rps, error_rate.
    """
    rng = np.random.default_rng(SEED + hash(step_label) % 10000)
    if start_ts is None:
        start_ts = datetime.utcnow()
    times = np.linspace(0, duration_s, n)
    ts = [start_ts + timedelta(seconds=float(t)) for t in times]

    # Latency with low-frequency drift and noise
    drift = 0.2 * np.sin(2 * np.pi * times / max(1.0, duration_s / 5))
    noise = rng.normal(0, 5.0, size=n)
    latency = np.clip(base_latency_ms * (1.0 + 0.01 * drift) + noise, 1.0, None)

    # Throughput with mild trend and anti-correlation to latency
    throughput = np.clip(base_rps * (1.0 - 0.002 * drift) + rng.normal(0, 2.0, size=n), 0.0, None)

    # Error rate baseline with rare spikes
    errors = np.clip(rng.binomial(1, error_p, size=n) * rng.uniform(0.2, 0.8, size=n), 0.0, 1.0)

    if spike_every:
        for i in range(n):
            if i % spike_every == 0 and i > 0:
                latency[i] *= rng.uniform(1.2, 1.6)
                throughput[i] *= rng.uniform(0.6, 0.9)
                errors[i] = max(errors[i], rng.uniform(0.05, 0.2))

    df = pd.DataFrame({
        "ts": ts,
        "t_s": times,
        "step": step_label,
        "latency_ms": latency,
        "throughput_rps": throughput,
        "error_rate": errors,
    })
    return df


# quick smoke
_sim = simulate_telemetry(n=50, step_label="smoke")
_sim.head()

## 4) Definir secuencias de interacción canónicas

Creamos dataclasses y un mini-DSL YAML/Dict para describir pasos (acción, duración, parámetros).
Incluimos tres casos canónicos: 'login-browse-purchase', 'search-filter-paginate', 'retry-on-error'.

@dataclass
class InteractionStep:
    action: str
    duration_s: int
    params: Dict[str, Any]


@dataclass
class InteractionSequence:
    name: str
    steps: list[InteractionStep]


def parse_sequences_from_yaml(yaml_text: str) -> list[InteractionSequence]:
    doc = yaml.safe_load(yaml_text)
    sequences: list[InteractionSequence] = []
    for entry in doc.get("sequences", []):
        steps = [InteractionStep(s["action"], int(s.get("duration_s", 60)), s.get("params", {})) for s in entry["steps"]]
        sequences.append(InteractionSequence(name=entry["name"], steps=steps))
    return sequences


CANONICAL_YAML = """
sequences:
  - name: login-browse-purchase
    steps:
      - action: login
        duration_s: 60
        params: { base_latency_ms: 180, base_rps: 30, error_p: 0.02 }
      - action: browse
        duration_s: 120
        params: { base_latency_ms: 120, base_rps: 60, error_p: 0.01 }
      - action: purchase
        duration_s: 90
        params: { base_latency_ms: 150, base_rps: 40, error_p: 0.03 }
  - name: search-filter-paginate
    steps:
      - action: search
        duration_s: 100
        params: { base_latency_ms: 140, base_rps: 55, error_p: 0.015 }
      - action: filter
        duration_s: 80
        params: { base_latency_ms: 160, base_rps: 45, error_p: 0.02 }
      - action: paginate
        duration_s: 140
        params: { base_latency_ms: 130, base_rps: 65, error_p: 0.012 }
  - name: retry-on-error
    steps:
      - action: call
        duration_s: 60
        params: { base_latency_ms: 110, base_rps: 70, error_p: 0.005 }
      - action: error
        duration_s: 30
        params: { base_latency_ms: 220, base_rps: 20, error_p: 0.15, spike_every: 10 }
      - action: retry
        duration_s: 60
        params: { base_latency_ms: 130, base_rps: 55, error_p: 0.02 }
"""

SEQUENCES = parse_sequences_from_yaml(CANONICAL_YAML)
[name for name in [s.name for s in SEQUENCES]]

## 5) Ejecutar secuencias y registrar telemetría

Ejecutamos cada paso contra el simulador y agregamos DataFrames con etiquetas y timestamps continuos.

def run_sequence(seq: InteractionSequence, start_ts: datetime | None = None) -> pd.DataFrame:
    frames: list[pd.DataFrame] = []
    cur_ts = start_ts or datetime.utcnow()
    t_offset = 0.0
    for step in seq.steps:
        params = dict(step.params)
        params.setdefault("duration_s", step.duration_s)
        df = simulate_telemetry(
            n=max(50, int(params["duration_s"]) // 1),
            base_latency_ms=float(params.get("base_latency_ms", 120.0)),
            base_rps=float(params.get("base_rps", 50.0)),
            error_p=float(params.get("error_p", 0.01)),
            spike_every=params.get("spike_every"),
            duration_s=int(params["duration_s"]),
            step_label=step.action,
            start_ts=cur_ts,
        )
        # Rebase t_s to cumulative timeline
        df["t_s"] = df["t_s"] + t_offset
        frames.append(df)
        # Advance current ts and offset
        cur_ts = df["ts"].iloc[-1]
        t_offset = float(df["t_s"].iloc[-1])
    out = pd.concat(frames, ignore_index=True)
    out["sequence"] = seq.name
    return out


# Example run
_df = run_sequence(SEQUENCES[0])
_df.head()

## 6) Graficar figuras/curvas de telemetría

Generamos curvas de latencia, throughput y tasa de errores; devolvemos objetos matplotlib Figure para exportación.

def plot_telemetry(df: pd.DataFrame, title: str = "") -> dict[str, plt.Figure]:
    figs: dict[str, plt.Figure] = {}
    sns.set_style("whitegrid")

    # Latency over time
    fig1, ax1 = plt.subplots(figsize=(9, 4))
    for step, g in df.groupby("step"):
        ax1.plot(g["t_s"], g["latency_ms"], label=step, alpha=0.9)
    ax1.set_title(title or f"Latency over time — {df['sequence'].iloc[0]}")
    ax1.set_xlabel("t [s]")
    ax1.set_ylabel("latency [ms]")
    ax1.legend()
    figs["latency"] = fig1

    # Throughput over time
    fig2, ax2 = plt.subplots(figsize=(9, 4))
    for step, g in df.groupby("step"):
        ax2.plot(g["t_s"], g["throughput_rps"], label=step, alpha=0.9)
    ax2.set_title(title or f"Throughput over time — {df['sequence'].iloc[0]}")
    ax2.set_xlabel("t [s]")
    ax2.set_ylabel("throughput [rps]")
    ax2.legend()
    figs["throughput"] = fig2

    # Error rate over time
    fig3, ax3 = plt.subplots(figsize=(9, 4))
    for step, g in df.groupby("step"):
        ax3.plot(g["t_s"], g["error_rate"], label=step, alpha=0.9)
    ax3.set_title(title or f"Error rate over time — {df['sequence'].iloc[0]}")
    ax3.set_xlabel("t [s]")
    ax3.set_ylabel("error rate [0..1]")
    ax3.legend()
    figs["errors"] = fig3

    # Distributions
    fig4, ax4 = plt.subplots(1, 3, figsize=(12, 3))
    sns.kdeplot(df["latency_ms"], fill=True, ax=ax4[0])
    ax4[0].set_title("Latency dist")
    sns.kdeplot(df["throughput_rps"], fill=True, ax=ax4[1])
    ax4[1].set_title("Throughput dist")
    sns.kdeplot(df["error_rate"], fill=True, ax=ax4[2])
    ax4[2].set_title("Error rate dist")
    fig4.suptitle(title or f"Distributions — {df['sequence'].iloc[0]}")
    figs["distributions"] = fig4

    plt.tight_layout()
    return figs


_figs = plot_telemetry(_df)
plt.close('all')  # Avoid duplicate inline rendering when exporting

## 7) Validar resultados y reproducibilidad

Calculamos hashes de parámetros/datos, registramos versión de dependencias y semilla, y afirmamos invariantes básicos.

def df_hash(df: pd.DataFrame) -> str:
    m = hashlib.sha256()
    # Stable hash: round floats; serialize selected columns
    cols = ["t_s", "latency_ms", "throughput_rps", "error_rate", "step", "sequence"]
    sub = df[cols].copy()
    for c in ["t_s", "latency_ms", "throughput_rps", "error_rate"]:
        sub[c] = pd.to_numeric(sub[c], errors="coerce").round(6)
    payload = sub.to_csv(index=False).encode("utf-8")
    m.update(payload)
    return m.hexdigest()


def validate_df(df: pd.DataFrame) -> dict[str, Any]:
    assert (df["t_s"].diff().fillna(0) >= 0).all(), "Timestamps must be non-decreasing"
    for c in ["latency_ms", "throughput_rps", "error_rate"]:
        assert (df[c] >= 0).all(), f"{c} must be non-negative"
    return {
        "rows": int(len(df)),
        "duration_s": float(df["t_s"].iloc[-1] - df["t_s"].iloc[0]) if len(df) > 1 else 0.0,
        "hash": df_hash(df),
    }


ENV_INFO = {
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "seed": SEED,
}
if pkg_resources is not None:
    try:
        ENV_INFO["numpy"] = pkg_resources.get_distribution("numpy").version
        ENV_INFO["pandas"] = pkg_resources.get_distribution("pandas").version
        ENV_INFO["matplotlib"] = pkg_resources.get_distribution("matplotlib").version
        ENV_INFO["seaborn"] = pkg_resources.get_distribution("seaborn").version
        ENV_INFO["pyyaml"] = pkg_resources.get_distribution("pyyaml").version
    except Exception:
        pass

ENV_INFO

## 8) Exportar figuras y artefactos a docs/assets/

Guardamos figuras (PNG/SVG), tablas (CSV) y un manifiesto con rutas y metadatos para cada secuencia.

def export_sequence(df: pd.DataFrame, figs: dict[str, plt.Figure], out_dir: Path) -> dict[str, Any]:
    out_dir.mkdir(parents=True, exist_ok=True)
    manifest: dict[str, Any] = {
        "sequence": df["sequence"].iloc[0],
        "artifacts": [],
    }

    # Save table
    csv_path = out_dir / "telemetry.csv"
    df.to_csv(csv_path, index=False)
    manifest["table_csv"] = str(csv_path.relative_to(DOCS_DIR)) if csv_path.is_relative_to(DOCS_DIR) else str(csv_path)

    # Save figures
    for key, fig in figs.items():
        png_path = out_dir / f"{key}.png"
        svg_path = out_dir / f"{key}.svg"
        fig.savefig(png_path, dpi=160, bbox_inches="tight")
        fig.savefig(svg_path, bbox_inches="tight")
        plt.close(fig)
        manifest["artifacts"].append({
            "name": key,
            "png": str(png_path.relative_to(DOCS_DIR)) if png_path.is_relative_to(DOCS_DIR) else str(png_path),
            "svg": str(svg_path.relative_to(DOCS_DIR)) if svg_path.is_relative_to(DOCS_DIR) else str(svg_path),
            "caption": f"{key.capitalize()} for {manifest['sequence']}",
            "alt": f"{key} plot for sequence {manifest['sequence']}",
        })

    # Meta
    manifest["summary"] = validate_df(df)
    manifest["env"] = ENV_INFO

    # Write manifest
    manifest_path = out_dir / "manifest.json"
    manifest_path.write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    manifest["manifest"] = str(manifest_path.relative_to(DOCS_DIR)) if manifest_path.is_relative_to(DOCS_DIR) else str(manifest_path)
    return manifest


# Demo export for first sequence (paths under docs/assets/interactions/<name>)
_demo_df = run_sequence(SEQUENCES[0])
_demo_figs = plot_telemetry(_demo_df)
_demo_out = ASSETS_DIR / SEQUENCES[0].name
_demo_manifest = export_sequence(_demo_df, _demo_figs, _demo_out)
_demo_manifest

## 9) Generar docs/INTERACTIONS_GUIDE.md

Escribimos el Markdown con índice de secuencias, imágenes y enlaces a CSV/JSON. Si ya existe, lo sobreescribimos (fuente canónica para esta guía).

def write_interactions_guide(manifests: list[dict[str, Any]], guide_path: Path | None = None) -> Path:
    guide = guide_path or (DOCS_DIR / "INTERACTIONS_GUIDE.md")
    lines: list[str] = []
    lines.append("# Interaction Sequences Guide")
    lines.append("")
    lines.append("This page is generated by notebooks/Interaction_Sequences.ipynb. It summarizes canonical interaction sequences, their telemetry, and links to CSV/JSON artifacts.")
    lines.append("")
    lines.append("## Sequences")
    for m in manifests:
        seq = m["sequence"]
        lines.append(f"### {seq}")
        # Summary table
        summ = m.get("summary", {})
        lines.append("- Rows: " + str(summ.get("rows", "?")))
        lines.append("- Duration [s]: " + str(round(summ.get("duration_s", 0.0), 2)))
        lines.append("- Hash: `" + str(summ.get("hash", "")) + "`")
        lines.append("")
        # Table link
        if "table_csv" in m:
            lines.append(f"Data (CSV): [{m['table_csv']}]({m['table_csv']})")
            lines.append("")
        # Images
        for art in m.get("artifacts", []):
            png = art.get("png")
            caption = art.get("caption", art.get("name", "figure"))
            alt = art.get("alt", caption)
            if png:
                lines.append(f"![{alt}]({png})")
                lines.append(f"<sub>{caption}</sub>")
                lines.append("")
        # Manifest link
        if "manifest" in m:
            lines.append(f"Manifest (JSON): [{m['manifest']}]({m['manifest']})")
        lines.append("")
    # Environment
    lines.append("## Environment")
    lines.append("```json")
    lines.append(json.dumps(ENV_INFO, indent=2))
    lines.append("```")

    guide.write_text("\n".join(lines), encoding="utf-8")
    return guide


# Write guide for demo manifest (full export will add all)
_guide_path = write_interactions_guide([_demo_manifest])
str(_guide_path)

## 10) Tarea de exportación: función CLI y tarea de VS Code

Creamos export_all(sequences) que orquesta ejecución y exportación, y un CLI simple. La tarea de VS Code invocará nbconvert para ejecutar este notebook y producir un HTML, mientras que este CLI produce artefactos bajo docs/assets/.

def export_all(sequences: list[InteractionSequence] | None = None) -> list[dict[str, Any]]:
    seqs = sequences or SEQUENCES
    manifests: list[dict[str, Any]] = []
    for seq in seqs:
        df = run_sequence(seq)
        figs = plot_telemetry(df)
        out_dir = ASSETS_DIR / seq.name
        m = export_sequence(df, figs, out_dir)
        manifests.append(m)
    write_interactions_guide(manifests)
    return manifests


if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description="Export interaction sequences artifacts and guide")
    parser.add_argument("--seq", dest="seq", default=None, help="Single sequence name to export")
    parser.add_argument("--all", dest="all", action="store_true", help="Export all sequences")
    args = parser.parse_args()

    if args.seq:
        chosen = [s for s in SEQUENCES if s.name == args.seq]
        if not chosen:
            raise SystemExit(f"Sequence not found: {args.seq}")
        export_all(chosen)
    else:
        export_all(SEQUENCES if args.all else SEQUENCES[:1])

## 11) Escribir pruebas mínimas y ejecutar en VS Code

Generamos un archivo de pruebas mínimo para validar invariantes. En VS Code, puedes ejecutar las pruebas con el Test Explorer o desde la terminal.

TESTS_DIR = REPO_ROOT / "tests"
TESTS_DIR.mkdir(exist_ok=True)
TEST_FILE = TESTS_DIR / "test_interactions_sequences_nb.py"
TEST_FILE.write_text(
    """
import pandas as pd
from pathlib import Path


def test_manifest_and_csv_exist():
    guide = Path('docs/INTERACTIONS_GUIDE.md')
    assert guide.exists(), 'Guide must exist after running the notebook export cells.'
    # Find at least one manifest and CSV under assets
    assets = Path('docs/assets/interactions')
    assert assets.exists(), 'Assets dir must exist.'
    found_manifest = False
    found_csv = False
    for p in assets.rglob('manifest.json'):
        found_manifest = True
    for p in assets.rglob('telemetry.csv'):
        df = pd.read_csv(p)
        assert not df.empty
        found_csv = True
    assert found_manifest, 'At least one manifest.json must be exported.'
    assert found_csv, 'At least one telemetry.csv must be exported.'
    """,
    encoding="utf-8"
)
print(f"Wrote test file: {TEST_FILE}")


## 1) Configurar entorno y rutas de proyecto

Detectamos la raíz del repositorio (git o heurística con pathlib), creamos carpetas de salida y parametrizamos rutas para activos (PNG/SVG/CSV/JSON).

In [1]:
from __future__ import annotations
import os
import sys
from pathlib import Path
from typing import Dict, Any

# Detect repo root via git, else fallback to current working directory
try:
    import subprocess
    root_guess = Path.cwd()
    git_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], cwd=root_guess).decode().strip()
    REPO_ROOT = Path(git_root)
except Exception:
    # Heurística: buscar pyproject.toml hacia arriba
    p = Path.cwd()
    while p != p.parent:
        if (p / "pyproject.toml").exists():
            REPO_ROOT = p
            break
        p = p.parent
    else:
        REPO_ROOT = Path.cwd()

# Ensure src is importable during interactive runs
SRC_DIR = REPO_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

DOCS_DIR = REPO_ROOT / "docs"
ASSETS_DIR = DOCS_DIR / "assets" / "interactions"
DATA_DIR = REPO_ROOT / "data"
REPORTS_DIR = REPO_ROOT / "results" / "reports"

for d in [DOCS_DIR, ASSETS_DIR, DATA_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Assets dir: {ASSETS_DIR}")

Repo root: C:\TNFR-Python-Engine
Assets dir: C:\TNFR-Python-Engine\docs\assets\interactions


## 2) Importar librerías y establecer semilla

Importamos librerías mínimas y fijamos semilla global para reproducibilidad.

In [2]:
import json
import yaml
import math
import hashlib
import platform
import random
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

try:
    # Optional: richer version reporting
    import pkg_resources  # type: ignore
except Exception:
    pkg_resources = None

# Fixed seeds for reproducibility
SEED = int(os.environ.get("INTERACTIONS_SEED", 1337))
np.random.seed(SEED)
random.seed(SEED)

print({"numpy_seed": SEED})

C:\Users\nuevo\AppData\Local\Temp\ipykernel_26752\4252616425.py:17: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources  # type: ignore


{'numpy_seed': 1337}


## 3) Cargar o simular telemetría base

Implementamos carga desde CSV/JSON en data/ y un simulador de series sintéticas (tiempo, latencia, throughput, errores) con ruido controlado por la semilla.

In [3]:
def load_telemetry(path: Path) -> pd.DataFrame:
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(path)
    if path.suffix.lower() == ".csv":
        return pd.read_csv(path)
    if path.suffix.lower() in {".json", ".jsonl"}:
        if path.suffix.lower() == ".jsonl":
            rows = [json.loads(line) for line in path.read_text(encoding="utf-8").splitlines() if line.strip()]
            return pd.DataFrame(rows)
        return pd.DataFrame(json.loads(path.read_text(encoding="utf-8")))
    raise ValueError(f"Unsupported telemetry format: {path.suffix}")


def simulate_telemetry(n: int = 300, base_latency_ms: float = 120.0, base_rps: float = 50.0,
                        error_p: float = 0.01, spike_every: int | None = 60,
                        duration_s: int = 300, step_label: str = "base",
                        start_ts: datetime | None = None) -> pd.DataFrame:
    """Generate synthetic telemetry with gentle trends and occasional spikes.
    Columns: ts, t_s, step, latency_ms, throughput_rps, error_rate.
    """
    rng = np.random.default_rng(SEED + hash(step_label) % 10000)
    if start_ts is None:
        start_ts = datetime.utcnow()
    times = np.linspace(0, duration_s, n)
    ts = [start_ts + timedelta(seconds=float(t)) for t in times]

    # Latency with low-frequency drift and noise
    drift = 0.2 * np.sin(2 * np.pi * times / max(1.0, duration_s / 5))
    noise = rng.normal(0, 5.0, size=n)
    latency = np.clip(base_latency_ms * (1.0 + 0.01 * drift) + noise, 1.0, None)

    # Throughput with mild trend and anti-correlation to latency
    throughput = np.clip(base_rps * (1.0 - 0.002 * drift) + rng.normal(0, 2.0, size=n), 0.0, None)

    # Error rate baseline with rare spikes
    errors = np.clip(rng.binomial(1, error_p, size=n) * rng.uniform(0.2, 0.8, size=n), 0.0, 1.0)

    if spike_every:
        for i in range(n):
            if i % spike_every == 0 and i > 0:
                latency[i] *= rng.uniform(1.2, 1.6)
                throughput[i] *= rng.uniform(0.6, 0.9)
                errors[i] = max(errors[i], rng.uniform(0.05, 0.2))

    df = pd.DataFrame({
        "ts": ts,
        "t_s": times,
        "step": step_label,
        "latency_ms": latency,
        "throughput_rps": throughput,
        "error_rate": errors,
    })
    return df


# quick smoke
_sim = simulate_telemetry(n=50, step_label="smoke")
_sim.head()

C:\Users\nuevo\AppData\Local\Temp\ipykernel_26752\2291027102.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_ts = datetime.utcnow()


,ts,t_s,step,latency_ms,throughput_rps,error_rate
0,2025-11-12 14:34:17.822803,0.000000,smoke,115.431990,45.512227,0.0
1,2025-11-12 14:34:23.945252,6.122449,smoke,122.828537,48.413510,0.0
2,2025-11-12 14:34:30.067701,12.244898,smoke,117.161813,50.533080,0.0
3,2025-11-12 14:34:36.190150,18.367347,smoke,115.127453,50.299730,0.0
4,2025-11-12 14:34:42.312599,24.489796,smoke,110.435493,55.100262,0.0


## 4) Definir secuencias de interacción canónicas

Creamos dataclasses y un mini-DSL YAML/Dict para describir pasos (acción, duración, parámetros). Incluimos casos canónicos.

In [4]:
from dataclasses import dataclass, asdict
from typing import Dict, Any

@dataclass
class InteractionStep:
    action: str
    duration_s: int
    params: Dict[str, Any]


@dataclass
class InteractionSequence:
    name: str
    steps: list[InteractionStep]


def parse_sequences_from_yaml(yaml_text: str) -> list[InteractionSequence]:
    doc = yaml.safe_load(yaml_text)
    sequences: list[InteractionSequence] = []
    for entry in doc.get("sequences", []):
        steps = [InteractionStep(s["action"], int(s.get("duration_s", 60)), s.get("params", {})) for s in entry["steps"]]
        sequences.append(InteractionSequence(name=entry["name"], steps=steps))
    return sequences


CANONICAL_YAML = """
sequences:
  - name: login-browse-purchase
    steps:
      - action: login
        duration_s: 60
        params: { base_latency_ms: 180, base_rps: 30, error_p: 0.02 }
      - action: browse
        duration_s: 120
        params: { base_latency_ms: 120, base_rps: 60, error_p: 0.01 }
      - action: purchase
        duration_s: 90
        params: { base_latency_ms: 150, base_rps: 40, error_p: 0.03 }
  - name: search-filter-paginate
    steps:
      - action: search
        duration_s: 100
        params: { base_latency_ms: 140, base_rps: 55, error_p: 0.015 }
      - action: filter
        duration_s: 80
        params: { base_latency_ms: 160, base_rps: 45, error_p: 0.02 }
      - action: paginate
        duration_s: 140
        params: { base_latency_ms: 130, base_rps: 65, error_p: 0.012 }
  - name: retry-on-error
    steps:
      - action: call
        duration_s: 60
        params: { base_latency_ms: 110, base_rps: 70, error_p: 0.005 }
      - action: error
        duration_s: 30
        params: { base_latency_ms: 220, base_rps: 20, error_p: 0.15, spike_every: 10 }
      - action: retry
        duration_s: 60
        params: { base_latency_ms: 130, base_rps: 55, error_p: 0.02 }
"""

SEQUENCES = parse_sequences_from_yaml(CANONICAL_YAML)
[name for name in [s.name for s in SEQUENCES]]

['login-browse-purchase', 'search-filter-paginate', 'retry-on-error']

## 5) Ejecutar secuencias y registrar telemetría

Ejecutamos cada paso contra el simulador y agregamos DataFrames con etiquetas y timestamps continuos.

In [5]:
def run_sequence(seq: InteractionSequence, start_ts: datetime | None = None) -> pd.DataFrame:
    frames: list[pd.DataFrame] = []
    cur_ts = start_ts or datetime.utcnow()
    t_offset = 0.0
    for step in seq.steps:
        params = dict(step.params)
        params.setdefault("duration_s", step.duration_s)
        df = simulate_telemetry(
            n=max(50, int(params["duration_s"]) // 1),
            base_latency_ms=float(params.get("base_latency_ms", 120.0)),
            base_rps=float(params.get("base_rps", 50.0)),
            error_p=float(params.get("error_p", 0.01)),
            spike_every=params.get("spike_every"),
            duration_s=int(params["duration_s"]),
            step_label=step.action,
            start_ts=cur_ts,
        )
        # Rebase t_s to cumulative timeline
        df["t_s"] = df["t_s"] + t_offset
        frames.append(df)
        # Advance current ts and offset
        cur_ts = df["ts"].iloc[-1]
        t_offset = float(df["t_s"].iloc[-1])
    out = pd.concat(frames, ignore_index=True)
    out["sequence"] = seq.name
    return out


# Example run
_df = run_sequence(SEQUENCES[0])
_df.head()

C:\Users\nuevo\AppData\Local\Temp\ipykernel_26752\1488497.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cur_ts = start_ts or datetime.utcnow()


,ts,t_s,step,latency_ms,throughput_rps,error_rate,sequence
0,2025-11-12 14:34:29.494867,0.000000,login,178.172871,31.055583,0.0,login-browse-purchase
1,2025-11-12 14:34:30.511816,1.016949,login,180.243478,29.782623,0.0,login-browse-purchase
2,2025-11-12 14:34:31.528765,2.033898,login,185.146571,30.964893,0.0,login-browse-purchase
3,2025-11-12 14:34:32.545714,3.050847,login,188.108166,30.398561,0.0,login-browse-purchase
4,2025-11-12 14:34:33.562664,4.067797,login,183.744614,29.335008,0.0,login-browse-purchase


## 6) Graficar figuras/curvas de telemetría

Generamos curvas de latencia, throughput y tasa de errores; devolvemos objetos matplotlib Figure para exportación.

In [6]:
def plot_telemetry(df: pd.DataFrame, title: str = "") -> dict[str, plt.Figure]:
    figs: dict[str, plt.Figure] = {}
    sns.set_style("whitegrid")

    # Latency over time
    fig1, ax1 = plt.subplots(figsize=(9, 4))
    for step, g in df.groupby("step"):
        ax1.plot(g["t_s"], g["latency_ms"], label=step, alpha=0.9)
    ax1.set_title(title or f"Latency over time — {df['sequence'].iloc[0]}")
    ax1.set_xlabel("t [s]")
    ax1.set_ylabel("latency [ms]")
    ax1.legend()
    figs["latency"] = fig1

    # Throughput over time
    fig2, ax2 = plt.subplots(figsize=(9, 4))
    for step, g in df.groupby("step"):
        ax2.plot(g["t_s"], g["throughput_rps"], label=step, alpha=0.9)
    ax2.set_title(title or f"Throughput over time — {df['sequence'].iloc[0]}")
    ax2.set_xlabel("t [s]")
    ax2.set_ylabel("throughput [rps]")
    ax2.legend()
    figs["throughput"] = fig2

    # Error rate over time
    fig3, ax3 = plt.subplots(figsize=(9, 4))
    for step, g in df.groupby("step"):
        ax3.plot(g["t_s"], g["error_rate"], label=step, alpha=0.9)
    ax3.set_title(title or f"Error rate over time — {df['sequence'].iloc[0]}")
    ax3.set_xlabel("t [s]")
    ax3.set_ylabel("error rate [0..1]")
    ax3.legend()
    figs["errors"] = fig3

    # Distributions
    fig4, ax4 = plt.subplots(1, 3, figsize=(12, 3))
    sns.kdeplot(df["latency_ms"], fill=True, ax=ax4[0])
    ax4[0].set_title("Latency dist")
    sns.kdeplot(df["throughput_rps"], fill=True, ax=ax4[1])
    ax4[1].set_title("Throughput dist")
    sns.kdeplot(df["error_rate"], fill=True, ax=ax4[2])
    ax4[2].set_title("Error rate dist")
    fig4.suptitle(title or f"Distributions — {df['sequence'].iloc[0]}")
    figs["distributions"] = fig4

    plt.tight_layout()
    return figs


_figs = plot_telemetry(_df)
plt.close('all')  # Avoid duplicate inline rendering when exporting

## 7) Validar resultados y reproducibilidad

Calculamos hashes de parámetros/datos, registramos versión de dependencias y semilla, y afirmamos invariantes básicos.

In [7]:
def df_hash(df: pd.DataFrame) -> str:
    m = hashlib.sha256()
    # Stable hash: round floats; serialize selected columns
    cols = ["t_s", "latency_ms", "throughput_rps", "error_rate", "step", "sequence"]
    sub = df[cols].copy()
    for c in ["t_s", "latency_ms", "throughput_rps", "error_rate"]:
        sub[c] = pd.to_numeric(sub[c], errors="coerce").round(6)
    payload = sub.to_csv(index=False).encode("utf-8")
    m.update(payload)
    return m.hexdigest()


def validate_df(df: pd.DataFrame) -> dict[str, Any]:
    assert (df["t_s"].diff().fillna(0) >= 0).all(), "Timestamps must be non-decreasing"
    for c in ["latency_ms", "throughput_rps", "error_rate"]:
        assert (df[c] >= 0).all(), f"{c} must be non-negative"
    return {
        "rows": int(len(df)),
        "duration_s": float(df["t_s"].iloc[-1] - df["t_s"].iloc[0]) if len(df) > 1 else 0.0,
        "hash": df_hash(df),
    }


ENV_INFO = {
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "seed": SEED,
}
if pkg_resources is not None:
    try:
        ENV_INFO["numpy"] = pkg_resources.get_distribution("numpy").version
        ENV_INFO["pandas"] = pkg_resources.get_distribution("pandas").version
        ENV_INFO["matplotlib"] = pkg_resources.get_distribution("matplotlib").version
        ENV_INFO["seaborn"] = pkg_resources.get_distribution("seaborn").version
        ENV_INFO["pyyaml"] = pkg_resources.get_distribution("pyyaml").version
    except Exception:
        pass

ENV_INFO

{'python': '3.13.6',
 'platform': 'Windows-11-10.0.26200-SP0',
 'seed': 1337,
 'numpy': '2.3.4',
 'pandas': '2.2.3',
 'matplotlib': '3.10.7',
 'seaborn': '0.13.2',
 'pyyaml': '6.0.2'}

## 8) Exportar figuras y artefactos a docs/assets/

Guardamos figuras (PNG/SVG), tablas (CSV) y un manifiesto con rutas y metadatos para cada secuencia.

In [12]:
def export_sequence(df: pd.DataFrame, figs: dict[str, plt.Figure], out_dir: Path) -> dict[str, Any]:
    out_dir.mkdir(parents=True, exist_ok=True)
    manifest: dict[str, Any] = {
        "sequence": df["sequence"].iloc[0],
        "artifacts": [],
    }

    # Save table
    csv_path = out_dir / "telemetry.csv"
    df.to_csv(csv_path, index=False)
    try:
        rel_csv = csv_path.relative_to(DOCS_DIR)
        manifest["table_csv"] = str(rel_csv)
    except ValueError:
        manifest["table_csv"] = str(csv_path)

    # Save figures
    for key, fig in figs.items():
        png_path = out_dir / f"{key}.png"
        svg_path = out_dir / f"{key}.svg"
        fig.savefig(png_path, dpi=160, bbox_inches="tight")
        fig.savefig(svg_path, bbox_inches="tight")
        plt.close(fig)
        try:
            rel_png = png_path.relative_to(DOCS_DIR)
            rel_svg = svg_path.relative_to(DOCS_DIR)
        except ValueError:
            rel_png = png_path
            rel_svg = svg_path
        manifest["artifacts"].append({
            "name": key,
            "png": str(rel_png),
            "svg": str(rel_svg),
            "caption": f"{key.capitalize()} for {manifest['sequence']}",
            "alt": f"{key} plot for sequence {manifest['sequence']}",
        })

    # Meta
    manifest["summary"] = validate_df(df)
    manifest["env"] = ENV_INFO

    # Write manifest
    manifest_path = out_dir / "manifest.json"
    manifest_path.write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    try:
        rel_manifest = manifest_path.relative_to(DOCS_DIR)
        manifest["manifest"] = str(rel_manifest)
    except ValueError:
        manifest["manifest"] = str(manifest_path)
    return manifest


# Export ALL sequences to docs/assets/interactions/<sequence>/ and build manifests
manifests: list[dict[str, Any]] = []
for _seq in SEQUENCES:
    _df = run_sequence(_seq)
    _figs = plot_telemetry(_df)
    _out = ASSETS_DIR / _seq.name
    _m = export_sequence(_df, _figs, _out)
    manifests.append(_m)

# Show a compact summary (sequence + rows)
[{"sequence": m.get("sequence"), "rows": m.get("summary", {}).get("rows")} for m in manifests]

C:\Users\nuevo\AppData\Local\Temp\ipykernel_26752\1488497.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cur_ts = start_ts or datetime.utcnow()
C:\Users\nuevo\AppData\Local\Temp\ipykernel_26752\1488497.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cur_ts = start_ts or datetime.utcnow()
C:\Users\nuevo\AppData\Local\Temp\ipykernel_26752\1488497.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cur_ts = start_ts or datetime.utcnow()


[{'sequence': 'login-browse-purchase', 'rows': 270},
 {'sequence': 'search-filter-paginate', 'rows': 320},
 {'sequence': 'retry-on-error', 'rows': 170}]

## 9) Generar docs/INTERACTIONS_GUIDE.md

Escribimos el Markdown con índice de secuencias, imágenes y enlaces a CSV/JSON. Si ya existe, se sobreescribe.

In [13]:
def write_interactions_guide(manifests: list[dict[str, Any]], guide_path: Path | None = None) -> Path:
    guide = guide_path or (DOCS_DIR / "INTERACTIONS_GUIDE.md")
    lines: list[str] = []
    lines.append("# Interaction Sequences Guide")
    lines.append("")
    lines.append("This page is generated by notebooks/Interaction_Sequences.ipynb. It summarizes canonical interaction sequences, their telemetry, and links to CSV/JSON artifacts.")
    lines.append("")
    lines.append("## Sequences")
    for m in manifests:
        seq = m["sequence"]
        lines.append(f"### {seq}")
        summ = m.get("summary", {})
        lines.append("- Rows: " + str(summ.get("rows", "?")))
        lines.append("- Duration [s]: " + str(round(summ.get("duration_s", 0.0), 2)))
        lines.append("- Hash: `" + str(summ.get("hash", "")) + "`")
        lines.append("")
        if "table_csv" in m:
            lines.append(f"Data (CSV): [{m['table_csv']}]({m['table_csv']})")
            lines.append("")
        for art in m.get("artifacts", []):
            png = art.get("png")
            caption = art.get("caption", art.get("name", "figure"))
            alt = art.get("alt", caption)
            if png:
                lines.append(f"![{alt}]({png})")
                lines.append(f"<sub>{caption}</sub>")
                lines.append("")
        if "manifest" in m:
            lines.append(f"Manifest (JSON): [{m['manifest']}]({m['manifest']})")
        lines.append("")
    lines.append("## Environment")
    lines.append("```json")
    lines.append(json.dumps(ENV_INFO, indent=2))
    lines.append("```")

    guide.write_text("\n".join(lines), encoding="utf-8")
    return guide


# Write guide for all exported manifests
_guide_path = write_interactions_guide(manifests)
str(_guide_path)

'C:\\TNFR-Python-Engine\\docs\\INTERACTIONS_GUIDE.md'

## 10) Tarea de exportación: función CLI y tarea de VS Code

Creamos export_all(sequences) que orquesta ejecución y exportación, y un CLI simple. La tarea de VS Code invocará nbconvert para ejecutar este notebook y producir un HTML.

In [ ]:
def export_all(sequences: list[InteractionSequence] | None = None) -> list[dict[str, Any]]:
    seqs = sequences or SEQUENCES
    manifests: list[dict[str, Any]] = []
    for seq in seqs:
        df = run_sequence(seq)
        figs = plot_telemetry(df)
        out_dir = ASSETS_DIR / seq.name
        m = export_sequence(df, figs, out_dir)
        manifests.append(m)
    write_interactions_guide(manifests)
    return manifests


if __name__ == "__main__":
    import argparse, sys
    parser = argparse.ArgumentParser(description="Export interaction sequences artifacts and guide")
    parser.add_argument("--seq", dest="seq", default=None, help="Single sequence name to export")
    parser.add_argument("--all", dest="all", action="store_true", help="Export all sequences")
    # In Jupyter/IPython, unknown args are present; use parse_known_args to avoid SystemExit
    args, _unknown = parser.parse_known_args()

    if args.seq:
        chosen = [s for s in SEQUENCES if s.name == args.seq]
        if not chosen:
            raise SystemExit(f"Sequence not found: {args.seq}")
        export_all(chosen)
    else:
        export_all(SEQUENCES if args.all else SEQUENCES[:1])

usage: ipykernel_launcher.py [-h] [--seq SEQ] [--all]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\nuevo\AppData\Roaming\jupyter\runtime\kernel-v3a1425fe1cd0da960601d316af915ea2f5a97153a.json


SystemExit: 2

C:\Users\nuevo\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 11) Escribir pruebas mínimas y ejecutar en VS Code

Generamos un archivo de pruebas mínimo para validar invariantes. En VS Code, puedes ejecutar las pruebas con el Test Explorer o desde la terminal.

In [11]:
TESTS_DIR = REPO_ROOT / "tests"
TESTS_DIR.mkdir(exist_ok=True)
TEST_FILE = TESTS_DIR / "test_interactions_sequences_nb.py"
TEST_FILE.write_text(
    """
import pandas as pd
from pathlib import Path


def test_manifest_and_csv_exist():
    guide = Path('docs/INTERACTIONS_GUIDE.md')
    assert guide.exists(), 'Guide must exist after running the notebook export cells.'
    # Find at least one manifest and CSV under assets
    assets = Path('docs/assets/interactions')
    assert assets.exists(), 'Assets dir must exist.'
    found_manifest = False
    found_csv = False
    for p in assets.rglob('manifest.json'):
        found_manifest = True
    for p in assets.rglob('telemetry.csv'):
        df = pd.read_csv(p)
        assert not df.empty
        found_csv = True
    assert found_manifest, 'At least one manifest.json must be exported.'
    assert found_csv, 'At least one telemetry.csv must be exported.'
    """,
    encoding="utf-8"
)
print(f"Wrote test file: {TEST_FILE}")

Wrote test file: C:\TNFR-Python-Engine\tests\test_interactions_sequences_nb.py
